In [1]:
# mv this notebook in src folder
# or
# set sys path to src folder
import os, sys
path_to_scr_folder=os.path.join(os.path.dirname(os.path.abspath('')), 'src')
sys.path.append(path_to_scr_folder)
from read_chroma import read_only_chroma, read_chromato_and_chromato_cube


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from projection import matrix_to_chromato, chromato_to_matrix
from matplotlib import cm
import time
import glob
from mass_spec import read_hmdb_spectrum
import pandas as pd
import seaborn as sn
import pandas as pd
def visualizer2(chromato_obj, mod_time = 1.25, rt1 = None, rt2 = None, rt1_window = 5, rt2_window = 0.1, plotly = False, title = "", points = None, radius=None, pt_shape = ".", log_chromato=True, casnos_dict=None, contour=[], center_pt=None, center_pt_window_1 = None, center_pt_window_2 = None, save=False):
    r"""Plot mass spectrum

    Parameters
    ----------
    chromato_obj :
        chromato_obj=(chromato, time_rn)
    rt1: optional
        Center the plot in the first dimension around rt1
    rt2: optional
        Center the plot in the second dimension around rt2
    rt1_window: optional
        If rt1, window size in the first dimension around rt1
    rt2_window: optional
        If rt2, window size in the second dimension around rt2
    points: optional
        Coordinates to displayed on the chromatogram
    radius: optional
        If points, dislay their radius (blobs detection)
    pt_shape: optional
        Shape of the points to be displayed.
    log_chromato: optional
        Apply logarithm function to the chromato before visualization
    contour: optional
        Displays stitch outlines
    center_pt: optional
        Center the plot around center_pt coordinates
    center_pt_window_1: optional
        If center_pt, window size in the first dimension around center_pt first coordinate
    center_pt_window_2: optional
        If center_pt, window size in the second dimension around center_pt second coordinate
    casnos_dict: optional
        If center_pt, window size in the second dimension around center_pt second coordinate
    title: optional
        Title of the plot
    Returns
    -------
    Examples
    --------
    >>> import plot
    >>> import read chroma
    >>> import utils
    >>> chromato_obj = read_chroma.read_chroma(filename, mod_time)
    >>> chromato,time_rn,spectra_obj = chromato_obj
    >>> matches = matching.matching_nist_lib(chromato_obj, spectra, some_pixel_coordinates)
    >>> casnos_dict = utils.get_name_dict(matches)
    >>> coordinates_in_time = projection.matrix_to_chromato(u,time_rn, mod_time,chromato.shape)
    >>> plot.visualizer(chromato_obj=(chromato, time_rn), mod_time=mod_time, points=coordinates_in_time, casnos_dict=casnos_dict)
    """
    chromato, time_rn = chromato_obj
    shape = chromato.shape
    X = np.linspace(time_rn[0], time_rn[1], shape[0])
    Y = np.linspace(0, mod_time, shape[1])
    if (rt1 is not None and rt2 is not None):
        rt1minusrt1window = rt1 - rt1_window
        rt1plusrt1window = rt1 + rt1_window
        rt2minusrt2window = rt2 - rt2_window
        rt2plusrt2window = rt2 + rt2_window
        if (rt1minusrt1window < time_rn[0]):
            rt1minusrt1window = time_rn[0]
            rt1plusrt1window = rt1 + rt1_window
        if (rt1plusrt1window > time_rn[1]):
            rt1plusrt1window = time_rn[1]
            rt1minusrt1window = rt1 - rt1_window
        if (rt2minusrt2window < 0):
            rt2minusrt2window = 0
            rt2plusrt2window = rt2 + rt2_window
        if (rt2plusrt2window > mod_time):
            rt2plusrt2window = mod_time
            rt2minusrt2window = rt2 - rt2_window
        position_in_chromato = np.array([[rt1minusrt1window, rt2minusrt2window], [rt1plusrt1window, rt2plusrt2window]])
        indexes = chromato_to_matrix(position_in_chromato,time_rn=time_rn, mod_time=mod_time, chromato_dim=shape)
        indexes_in_chromato = matrix_to_chromato(indexes,time_rn=time_rn, mod_time=mod_time, chromato_dim=shape)
        chromato = chromato[indexes[0][0]:indexes[1][0], indexes[0][1]:indexes[1][1]]
        X = np.linspace(rt1minusrt1window, rt1plusrt1window, indexes[1][0] - indexes[0][0])
        Y = np.linspace(rt2minusrt2window, rt2plusrt2window, indexes[1][1] - indexes[0][1])
    elif (center_pt_window_1 and center_pt_window_2):
        center_pt1_minusrt1window = center_pt[0] - center_pt_window_1
        center_pt1_plusrt1window =  center_pt[0] + center_pt_window_1
        center_pt2_minusrt2window =  center_pt[1] - center_pt_window_2
        center_pt2_plusrt2window =  center_pt[1] + center_pt_window_2
        if (center_pt1_minusrt1window < 0):
            center_pt1_minusrt1window = 0
            center_pt1_plusrt1window = 2 * center_pt[0]
        if (center_pt1_plusrt1window >= shape[0]):
            center_pt1_plusrt1window = shape[0] - 1
            center_pt1_minusrt1window = center_pt[0] - abs(center_pt[0] - center_pt1_plusrt1window)
        if (center_pt2_minusrt2window < 0):
            center_pt2_minusrt2window = 0
            center_pt2_plusrt2window = 2 * center_pt[1]
        if (center_pt2_plusrt2window >= shape[1]):
            center_pt2_plusrt2window = shape[1] - 1
            center_pt2_minusrt2window = center_pt[1] - abs(center_pt[1] - center_pt2_plusrt2window)

        chromato = chromato[center_pt1_minusrt1window:center_pt1_plusrt1window + 1, center_pt2_minusrt2window:center_pt2_plusrt2window + 1]
        position_in_chromato = np.array([[center_pt1_minusrt1window, center_pt2_minusrt2window], [center_pt1_plusrt1window, center_pt2_plusrt2window]])
        indexes = matrix_to_chromato(position_in_chromato,time_rn=time_rn, mod_time=mod_time, chromato_dim=shape)
        #indexes_in_chromato = matrix_to_chromato(indexes,time_rn=time_rn, mod_time=mod_time, chromato_dim=shape)
        indexes_in_chromato=indexes

        X = np.linspace(indexes[0][0], indexes[1][0], chromato.shape[0])
        Y = np.linspace(indexes[0][1], indexes[1][1], chromato.shape[1])

        indexes = np.array([[center_pt1_minusrt1window, center_pt2_minusrt2window], [center_pt1_plusrt1window + 1, center_pt2_plusrt2window + 1]])
    if (log_chromato):
        chromato = np.log(chromato)
    chromato = np.transpose(chromato)
    fig, ax = plt.subplots()

    #tmp = ax.pcolormesh(X, Y, chromato)
    tmp = ax.contourf(X, Y, chromato)
    plt.colorbar(tmp)
    if (title != ""):
        plt.title(title)
    if (points is not None):
        if ((rt1 and rt2) or (center_pt_window_1 and center_pt_window_2)):
            tmp = []
            point_indexes = chromato_to_matrix(points,time_rn=time_rn, mod_time=mod_time, chromato_dim=shape)
            for i, point in enumerate(point_indexes):
                if (point_is_visible(point, indexes)):
                    tmp.append(points[i])

            points = np.array(tmp)
        if (radius is not None and len(points) > 0):
            for i in range(len(points)):
                c = plt.Circle((points[i][0], points[i][1]), radius[i] / shape[1] , color="red", linewidth=2, fill=False)
                ax.add_patch(c)
        if (len(points) > 0):
            if (casnos_dict != None):
                mol_name = []
                scatter_list = []
                comp_list = list(casnos_dict.keys())
                nb_comp = len(comp_list)
                cmap = get_cmap(nb_comp)
                for i, casno in enumerate(comp_list):
                    tmp_pt_list = []
                    for pt in casnos_dict[casno]:
                        if (not((rt1 and rt2) or (center_pt_window_1 and center_pt_window_2)) or point_is_visible(pt, indexes_in_chromato)):
                            print(casno)
                            tmp_pt_list.append(pt)
                    '''x_pts = np.array(casnos_dict[casno])[:,0]
                    y_pts = np.array(casnos_dict[casno])[:,1]'''
                    if len(tmp_pt_list) == 0:
                        continue
                    print("----")

                    mol_name.append(comp_list[i])
                    tmp_pt_list = np.array(tmp_pt_list)
                    x_pts = tmp_pt_list[:,0]
                    y_pts = tmp_pt_list[:,1]
                    tmp = ax.scatter(x_pts,y_pts, c=cmap(i), marker=pt_shape, cmap='hsv')
                    scatter_list.append(tmp)
                print(mol_name)
                plt.legend(scatter_list,
                    mol_name,
                    scatterpoints=1, fontsize=8, ncol=1, bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left",
                mode="expand")
            else:
                ax.plot(points[:,0], points[:,1], "r" + pt_shape)
    if (len(contour)):
        if (center_pt_window_1 and center_pt_window_2):
            indexes_in_chromato = matrix_to_chromato(indexes,time_rn=time_rn, mod_time=mod_time, chromato_dim=shape)
            tmp = []
            for i in range(len(contour)):
                if (point_is_visible(contour[i], indexes_in_chromato)):
                    tmp.append(contour[i])
            tmp=np.array(tmp)
            ax.plot(tmp[:,0], tmp[:,1], "b.")
        else:
            ax.plot(contour[:,0], contour[:,1], "b.")
    if (save):
        plt.savefig("figs/chromato_" + title + ".png")
    plt

In [ ]:
import numpy as np
path= "D:/GCxGC_MS/DATA/Dossier_partagé_GCxGC/Manue/"
dirs=['2025_06_27_GCxGC_VOLATIL_CF_08bis_postPTR/']

all_files = []
for dirnames in dirs:
    filenames= os.listdir(path+dirnames)
    for filename in filenames:
        all_files.append(path+dirnames+filename)
all_files
files=[file for file in all_files if ('.cdf' in file)]
QCfiles=[file for file in files if ('QC' in file)]

files


In [25]:
import numpy as np
path= "D:/GCxGC_MS/DATA/Dossier_partagé_GCxGC/Manue/2025_06_27_GCxGC_VOLATIL_CF_08bis_postPTR/"
files = os.listdir(path)
files

['15-04-25_817822_QC_23newEI.cdf',
 '2025-04-10-854514_QC.cdf',
 '2025-04-16_751318_QCnew23EI.cdf',
 '2025-04-16_751321_QCnew23EI.cdf',
 '2025-04-24_751325_QC-23-EIs.cdf',
 '2025-05-14_817827-QC_23EI_prep22-04.cdf',
 '2025-05-22_854516_QC_23EI_postGC_prep15mai25.cdf',
 '751301_27oct23_Blk_postPTR_split5.cdf',
 '751303_0038LJ_M1_postPTR_split5.cdf',
 '751304_0049VA_J0_postPTR_split5.cdf',
 '751308_0040AK_J0_postPTR_split5.cdf',
 '751310_0048GL_M1_postPTR_split5.cdf',
 '751315_0033CN_J7_postPTR_split5.cdf',
 '751316_0038LJ_J0_postPTR_split5.cdf',
 '751320_0045MO_M3_postPTR_split5.cdf',
 '751323_02avr24_Blk_postPTR_split5.cdf',
 '751325_QC_12EI_post_condi.cdf',
 '751329_0035DT_M3_postPTR_split5.cdf',
 '751330_24avr24_Blk_postPTR_split5.cdf',
 '751332_0041LM_M3_postPTR_split5.cdf',
 '751333_0039LL_J0unsure_postPTR_split5.cdf',
 '751334_0037TP_M6_postPTR_split5.cdf',
 '751335_0035DT_J0_postPTR_split5.cdf',
 '751340_14nov23_Blk_postPTR_split5.cdf',
 '751342_0052MM_J7_postPTR_split5.cdf',
 '7

In [27]:
df

,Unnamed: 0,751303_0038LJ_M1_postPTR_split.cdf,751304_0049VA_J0_postPTR_split.cdf,751308_0040AK_J0_postPTR_split.cdf,751310_0048GL_M1_postPTR_split.cdf,751315_0033CN_J7_postPTR_split.cdf,751316_0038LJ_J0_postPTR_split.cdf,751320_0045MO_M3_postPTR_split.cdf,751329_0035DT_M3_postPTR_split.cdf,751332_0041LM_M3_postPTR_split.cdf,...,854519_0043TG_M6_postPTR_split.cdf,854520_0037TP_J0_postPTR_split.cdf,854525_0040AK_M3_postPTR_split.cdf,854526_0041LM_J7_postPTR_split.cdf,854528_0046DE_J0_postPTR_split.cdf,854530_0045MO_M6_postPTR_split.cdf,864682_0044DA_J0bis_postPTR_split.cdf,864683_0044DA_J7_postPTR_split.cdf,864684_0046DE_J7_postPTR_split.cdf,864688_0033CN_M1_postPTR_split.cdf
0,Analyte1,"470.018493651442 , 1.0230088495575222","470.018493651442 , 0.9979351032448378","470.018493651442 , 1.017994100294985","470.018493651442 , 1.0029498525073746","470.018493651442 , 1.017994100294985","470.018493651442 , 1.0129793510324483","470.018493651442 , 1.0129793510324483","470.018493651442 , 1.0079646017699115","470.018493651442 , 1.0079646017699115",...,"470.018493651442 , 0.9979351032448378","470.018493651442 , 1.0029498525073746","470.018493651442 , 1.0029498525073746","470.018493651442 , 1.0129793510324483","470.018493651442 , 1.0029498525073746","470.018493651442 , 1.0029498525073746","470.018493651442 , 1.0079646017699115","470.018493651442 , 1.0129793510324483","470.018493651442 , 0.9979351032448378","470.018493651442 , 1.0129793510324483"
1,Analyte2,"1393.83607941449 , 0.9929203539823008","1393.83607941449 , 0.9728613569321535","1393.83607941449 , 0.9728613569321535","1393.83607941449 , 0.9728613569321535","1393.83607941449 , 0.987905604719764","1393.83607941449 , 0.9828908554572271","1393.83607941449 , 0.9828908554572271","1393.83607941449 , 0.9828908554572271","1393.83607941449 , 0.9828908554572271",...,"1393.83607941449 , 0.9678466076696164","1393.83607941449 , 0.9778761061946902","1393.83607941449 , 0.9778761061946902","1393.83607941449 , 0.9828908554572271","1393.83607941449 , 0.9778761061946902","1393.83607941449 , 0.9778761061946902","1393.83607941449 , 0.9828908554572271","1393.83607941449 , 0.987905604719764","1393.83607941449 , 0.9678466076696164","1393.83607941449 , 0.987905604719764"
2,Analyte3,"718.411435679813 , 0.005014749262536873","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0",...,"718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0","718.411435679813 , 0.0"
3,Analyte4,"716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7",...,"716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7","716.71011415907 , 1.7"
4,Analyte5,"764.347116739854 , 1.063126843657817","764.347116739854 , 1.0430678466076695","764.347116739854 , 1.0480825958702065","764.347116739854 , 1.0480825958702065","764.347116739854 , 1.0581120943952802","764.347116739854 , 1.0530973451327434","764.347116739854 , 1.0581120943952802","764.347116739854 , 1.0530973451327434","764.347116739854 , 1.0480825958702065",...,"764.347116739854 , 1.0430678466076695","764.347116739854 , 1.0530973451327434","764.347116739854 , 1.0480825958702065","764.347116739854 , 1.0581120943952802","764.347116739854 , 1.0480825958702065","764.347116739854 , 1.0530973451327434","764.347116739854 , 1.0480825958702065","764.347116739854 , 1.0581120943952802","764.347116739854 , 1.0430678466076695","764.347116739854 , 1.0581120943952802"
...,...,...,...,...

In [28]:
df = pd.read_csv("D:/GCxGC_MS/DATA/h5/2025_06_27_GCxGC_VOLATIL_CF_08bis_postPTR/resultPersistantHomology_tic/Align_table_RT.csv",sep=';', decimal=',')
df.shape[0]
df

#³files=[file for file in files if ('.cdf' in file)]
matching_files = [f for f in files if f in df.columns]
matching_files 

['751303_0038LJ_M1_postPTR_split5.cdf',
 '751304_0049VA_J0_postPTR_split5.cdf',
 '751308_0040AK_J0_postPTR_split5.cdf',
 '751310_0048GL_M1_postPTR_split5.cdf',
 '751315_0033CN_J7_postPTR_split5.cdf',
 '751316_0038LJ_J0_postPTR_split5.cdf',
 '751320_0045MO_M3_postPTR_split5.cdf',
 '751329_0035DT_M3_postPTR_split5.cdf',
 '751332_0041LM_M3_postPTR_split5.cdf',
 '751333_0039LL_J0unsure_postPTR_split5.cdf',
 '751334_0037TP_M6_postPTR_split5.cdf',
 '751335_0035DT_J0_postPTR_split5.cdf',
 '751342_0052MM_J7_postPTR_split5.cdf',
 '751346_0039LL_M1_postPTR_split5.cdf',
 '751347_0039LL_J0bis_postPTR_split5.cdf',
 '751351_0041LM_M6_postPTR_split5.cdf',
 '751353_0048GL_M3_postPTR_split5.cdf',
 '751354_0042MA_J0_postPTR_split5.cdf',
 '751355_0042MA_M3_postPTR_split5.cdf',
 '751357_0042MA_J7_postPTR_split5.cdf',
 '801834_inconnu_postPTR_split5.cdf',
 '801835_0050RI_M1_postPTR_split5.cdf',
 '801842_0032LC_M3_postPTR_split5.cdf',
 '801843_0036RB_M6_postPTR_split5.cdf',
 '801845_0044DA_M3_postPTR_split5

In [ ]:
"D:/GCxGC_MS/DATA/Dossier_partagé_GCxGC/Manue/GCxGC_VOLATIL-CF_01/751305_12QCd20_split10.txt"
QCfiles[0].replace(".cdf",".txt")

In [ ]:
pdf.close()

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
import matplotlib.backends.backend_pdf


for i in range(df.shape[0]):
    name=df.iloc[i,0]

    pdf = matplotlib.backends.backend_pdf.PdfPages("D:/GCxGC_MS/DATA/Dossier_partagé_GCxGC/Manue/res_python2dgc/plot/"+ name+".pdf")

    for file in QCfiles:
        s=df.loc[i,file]
        parts = s.replace('"', '').strip().split(',')
        rt1, rt2 = map(float, parts)
        chromato, time_rn= read_only_chroma(file, mod_time = 1.7)
        #fig = plt.figure()
        fig= visualizer2((chromato, time_rn ), log_chromato=True, mod_time=1.7, rt1= rt1/60	,rt2=rt2, rt1_window=0.3, rt2_window=0.3)
        plt.title(file)
        pdf.savefig(fig)
        plt.close(fig) 
    pdf.close()

In [30]:
plt.rcParams['figure.figsize'] = [7, 5]
import matplotlib.backends.backend_pdf


for i in range(15):
    name=df.iloc[i,0]

    pdf = matplotlib.backends.backend_pdf.PdfPages("D:/GCxGC_MS/DATA/h5/2025_06_27_GCxGC_VOLATIL_CF_08bis_postPTR/resultPersistantHomology_tic/plot"+ name+".pdf")

    for file in matching_files:
        s=df.loc[i,file]
        if pd.isna(s):
            continue
        parts = s.replace('"', '').strip().split(',')
        rt1, rt2 = map(float, parts)
        chromato, time_rn= read_only_chroma(path+file, mod_time = 1.7)
        #fig = plt.figure()
        fig= visualizer2((chromato, time_rn ), log_chromato=True, mod_time=1.7, rt1= rt1/60	,rt2=rt2, rt1_window=0.3, rt2_window=0.3)
        plt.title(file)
        pdf.savefig(fig)
        plt.close(fig) 
    pdf.close()